In [12]:
# importeren datasets
import pandas as pd

user_ratings = pd.read_csv('user_ratings.csv')
games = pd.read_csv('games.csv')

user_ratings.shape

user_ratings.head()


,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP


In [17]:
# opschonen dataset user_ratings
# ------------------------------------------------------------------

# Om er zeker van te zijn dat er geen ratings boven de 10 of onder de 0 zitten
filtered_user_ratings = user_ratings[(user_ratings['Rating'] >= 0) & (user_ratings['Rating'] <= 10)]

# user_ratings gegroupeerd op basis van de gemiddelde rating per game
group_user_rating = filtered_user_ratings.groupby('BGGId')[['Rating']].mean()
group_user_rating.reset_index()


,BGGId,Rating
0,1,7.672014
1,2,6.642697
2,3,7.484350
3,4,6.599476
4,5,7.439818
...,...,...
21920,347146,7.326190
21921,347521,7.790845
21922,348955,6.820896
21923,349131,6.465152


In [19]:
# opschonen dataset games
# --------------------------------------------------

# Niet-correcte jaartallen gefilterd
filtered_games = games[(games['YearPublished'] >= 400) & (games['YearPublished'] <= 2021)]

filtered_games.head()

# Game ID
# Year Published
# Avg Rating
# Avg Ranking per game

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.57979,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.45440,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.18227,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.23129,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.33583,2,6,...,21926,21926,0,1,0,0,0,0,0,0
